In [1]:
# Import libs
import findspark
findspark.init("/home/antonis/spark-2.3.0-bin-hadoop2.7")

import os.path
import pandas
import math
import time

from metrics.Correlation import Correlation
from IO.Write import Write
from IO.Read import Read
from metrics.RSquare import RSquare
from DataSet.Dataset import DataSet

import pyspark
from pyspark.sql import SparkSession


import sys
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql import Row
from pyspark.sql.functions import countDistinct,avg,stddev
from pyspark.ml.feature import (VectorAssembler,VectorIndexer,
                               OneHotEncoder , StringIndexer)

In [2]:
from pyspark import SparkConf, SparkContext
conf = (SparkConf()
    .set("spark.driver.maxResultSize", "20g")
       .set('spark_executor_cores',"3"))

In [3]:
#spark = SparkSession.builder.appName('melanoma').getOrCreate()
sc = SparkContext(conf=conf)
spark = SQLContext(sc)

In [ ]:
#spark.stop()

In [ ]:
#path = '/home/george/diplwmatikh/code/maf/maf = 0.05/assocs/'
#pathPatients = '/home/george/diplwmatikh/data/patients/Patients.txt'
pathSnp = '/media/antonis/red/newSet/maf/maf = 0.05/assoc/pvalue = 0.001/snp1.txt'

# Read ASSOCS

In [ ]:

start_time = time.time()
#path = '/home/george/snp/'
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)
log_txt = sc.textFile(path)
sc.setLogLevel("WARN")
header = log_txt.first()
log_txt = log_txt.filter(lambda line: line != header)
temp_var = log_txt.map(lambda k: k.split())

inferRow = temp_var.map(lambda p: Row(CHR=(p[0].strip()),SNP=p[1].strip(),BP=p[2].strip(),A1=p[3].strip(),F_A=p[4].strip()
                                   ,F_U=p[5].strip(),A2=p[6].strip(),CHISQ=p[7].strip(),P=(p[8].strip())
                                      ,OR=p[9].strip()))
schemaString = "CHR SNP BP A1 F_A F_U A2 CHISQ P OR"

fields = [StructField(field_name, StringType(), True) for field_name in schemaString.split()]
schema = StructType(fields)
schemaAssocs = spark.createDataFrame(inferRow,schema)
schemaAssocs.createOrReplaceTempView("Assocs")
#df=temp_var.toDF(header.split())
#log_df.select(log_df['field2']).show()
#log_df.filter(log_df['field2']=='1').count()
elapsed_time = time.time() - start_time
print(elapsed_time)
#df =spark.read.csv(path,header=True,inferSchema=True,escape="\t")

# Read Patients

In [ ]:
start_time = time.time()
#path = '/home/george/snp/'
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)
log_txt = sc.textFile(pathPatients)
sc.setLogLevel("WARN")
header = log_txt.first()
log_txt = log_txt.filter(lambda line: line != header)
temp_var = log_txt.map(lambda k: k.split())

inferRow = temp_var.map(lambda p: Row(eid=(p[0].strip()),sex=p[1].strip(),birth_year=p[2].strip(),cases=p[3].strip()))
schemaString = "eid sex birth_year cases"

fields = [StructField(field_name, StringType(), True) for field_name in schemaString.split()]
schema = StructType(fields)
schemaPatients = spark.createDataFrame(inferRow,schema)
schemaPatients.createOrReplaceTempView("Patients")
#df=temp_var.toDF(header.split())
#log_df.select(log_df['field2']).show()
#log_df.filter(log_df['field2']=='1').count()
elapsed_time = time.time() - start_time
print(elapsed_time)
#df =spark.read.csv(path,header=True,inferSchema=True,escape="\t")

# TO REMOVE NA

In [ ]:
schemaAssocs = schemaAssocs.where("OR != 'NA'").drop()

In [ ]:
schemaAssocs.createOrReplaceTempView("Assocs")


In [ ]:
sqlDF = spark.sql("SELECT * FROM Assocs WHERE SNP=='.' ")
sqlDF.show()

In [ ]:
schemaPatients.show()

# STRING INDEXING

In [ ]:
snp_indexer = StringIndexer(inputCol='SNP',outputCol='SNPIndex')
id_indexer = StringIndexer(inputCol='eid',outputCol='eidIndex')

In [ ]:
Patients =id_indexer.fit(schemaPatients).transform(schemaPatients)

In [ ]:
Patients.show()

In [ ]:
Assocs = snp_indexer.fit(schemaAssocs).transform(schemaAssocs)

In [ ]:
Assocs.show()

# GET IDS FROM PATINETS AND SNP

In [ ]:
def setIdToName(aList):
    
    ids = {}
    nameToId = {}
    idToName = {}
    count = 0
    
    for i in aList:
        
        nameToId[i] = count
        idToName[count] = i
        count += 1
        
    ids['nameToId'] = nameToId
    ids['idToName'] = idToName
    
    yield ids

In [ ]:
ids={}
test= Patients.select('eid')
#idss = test.toPandas().to_dict(orient='list')
snps = Assocs.select('SNPIndex')
idsa = snps.toPandas().to_dict(orient='list')
ids['patients'] = setIdToName(test.toPandas().to_dict(orient='list')['eid'])
ids['snps'] = idsa['SNPIndex']

In [ ]:
X, Y = read.readSnpsCode(patients,ids)

In [ ]:
test.toPandas().to_dict(orient='list')['eid']

# TESTS

In [4]:
#pathSnp = '/media/antonis/red/newSet/maf/maf = 0.05/assoc/pvalue = 0.001/snp1.txt'
pathSnp = '/media/antonis/red/newdata/maf = 0.05/pvalue = 0.001/snp2.txt'

#pathSnp = '/media/antonis/Antonis_Moulopoulos/newSet/pvalue = 0.001/snp1.txt'
#pathSnp = '/media/antonis/Antonis_Moulopoulos/newdata/maf = 0.05/pvalue = 0.001/snp2.txt'

data = spark.read.csv(pathSnp,inferSchema=True,header=True)
data=data.withColumnRenamed('TARGET','label')

In [5]:
features = []
for i in data.columns:
    if 'rs'  in i:
        features.append(i)
print(len(features))

12002


In [11]:
data.columns[len(data.columns)-1]

'label'

In [12]:
data.select('patients','label').show(10)

+--------+-----+
|patients|label|
+--------+-----+
|20480890| null|
|32535401| null|
|25233342| null|
|50043600| null|
|21612091| null|
|45058371| null|
|23464171| null|
|53582540| null|
|55526580| null|
|33210700| null|
+--------+-----+
only showing top 10 rows



In [13]:
data.select('label').show(10)

+-----+
|label|
+-----+
| null|
| null|
| null|
| null|
| null|
| null|
| null|
| null|
| null|
| null|
+-----+
only showing top 10 rows



In [7]:
assempler = VectorAssembler(inputCols=features,outputCol='features')
output = assempler.transform(data)
final_data = output.select('features','label')
#final_data.show(10)

# CORRELATION

In [8]:

from pyspark.ml.stat import Correlation
final_data1 = final_data.select('features')
corr = Correlation.corr(final_data1, "features")
corr = corr.head()[0].toArray()


In [15]:
snpsRed = []
snpsRed = s.lowCorrelation(corr, threshold=0.7, up=100, down=99)


count =  9470
len snpsRed =  9470


# CHI-SQUARE


In [ ]:
'''from pyspark.ml.stat import ChiSquareTest
from pyspark.ml.linalg import Vectors
data1 = [(1.0, Vectors.dense(23.0, 5.0)),
        (0.0, Vectors.dense(8.0, 10.0))]
df = spark.createDataFrame(data1, ["label", "features"])

r1 = ChiSquareTest.test(df, "features", "label").head()

'''r = ChiSquareTest.test(final_data, "features", "label").head()'''
print("pValues: " + str(r1.pValues))
print("degreesOfFreedom: " + str(r1.degreesOfFreedom))
print("statistics: " + str(r1.statistics))'''

In [ ]:
'''count = 0 
j= 0 
for i in r[0].toArray():
    if i >= 0.05:
        print(features[j])
        count += 1
    j += 1
print("count = ",count)'''

In [17]:
snpsRed1 = []
for i in snpsRed:
    snpsRed1.append(features[i])

assempler = VectorAssembler(inputCols=snpsRed1,outputCol='snpsRed')
output = assempler.transform(data)
final_data = output.select('snpsRed','label')
#final_data.show(10)

In [18]:
len(snpsRed1)

9470

In [19]:

train_data,test_data = final_data.randomSplit([0.9,0.1],seed=11)

train_data=train_data.withColumnRenamed('TARGET','label')
test_data=test_data.withColumnRenamed('TARGET','label')

In [20]:
print("final = ",final_data.count())
print("test_data = ", test_data.count())
print("train_data = ", train_data.count())
print("all = ", test_data.count() + train_data.count())

final =  242176
test_data =  24214
train_data =  217962
all =  242176


In [21]:
log_reg = LogisticRegression(featuresCol='snpsRed',labelCol='label',maxIter=10)
lr_model = log_reg.fit(train_data)

In [22]:
results = lr_model.transform(test_data)
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluate = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='label')
AUC = evaluate.evaluate(results)
AUC

1.0

In [23]:
'''Param for metric name in evaluation. Supports: - "rmse" (default): root mean squared error - 
"mse": mean squared error - "r2": R^2^ metric - "mae": mean absolute error '''

from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

#lr = LinearRegression(maxIter=maxIteration)
#pipeline = Pipeline(stages=[output, log_reg])
pipeline = Pipeline(stages=[log_reg])
#fit_model = pipeline.fit(train_data)
modelEvaluator=RegressionEvaluator(predictionCol='prediction', labelCol='label')
paramGrid = ParamGridBuilder().build()

crossval = CrossValidator(estimator=log_reg,
                          estimatorParamMaps=paramGrid,
                          evaluator=modelEvaluator,
                          numFolds=10)

cvModel = crossval.fit(train_data)
cross_results = cvModel.transform(test_data)
results = cross_results.select('prediction','label')
#results.show(10)

In [24]:
cross_results

DataFrame[snpsRed: vector, label: int, rawPrediction: vector, probability: vector, prediction: double]

In [25]:
from pyspark.mllib.evaluation import MulticlassMetrics


c = cross_results.rdd
predictionAndLabels = c.map(lambda lp: (lp.prediction, float(lp.label)))
metrics = MulticlassMetrics(predictionAndLabels)
metrics.confusionMatrix()

DenseMatrix(2, 2, [23988.0, 221.0, 4.0, 1.0], 0)

In [26]:
metrics.confusionMatrix().toArray()

array([[2.3988e+04, 4.0000e+00],
       [2.2100e+02, 1.0000e+00]])

In [32]:
print("accuracy = ", metrics.accuracy)
print("recal = ", metrics.recall(2))
print("precision = ", metrics.precision(2))
print("f1 = ", metrics.fMeasure(1.0))



accuracy =  0.9907078549599405
recal =  0.0045045045045045045
precision =  0.2
f1 =  0.9953320470529656


In [28]:
metrics.weightedPrecision



0.9836202592506461

In [29]:
metrics.weightedFMeasure()

0.9862873717696917

In [37]:
r = results.rdd

In [43]:
r1 = r.toDF()

In [52]:
t= r1.freqItems(['prediction'])

In [58]:
r3 = r.collect()

In [63]:
same1 = 0
same2 = 0
sum1 = 0
sum2 = 0
sumall = 0
for i in r3:
    
    if i[1] == 1:
        sum1 += 1
        if i[0] == 1:
            same1 += 1
    elif i[1] == 2:
        sum2 += 1
        if i[0] == 2:
            same2 += 1
    sumall += 1

In [64]:
print('sum1 = ', sum1)
print('sum2 = ', sum2)
print('same1 = ', same1)
print('same2 = ', same2)
print('all = ', sumall)
print('all2 = ', sum1+sum2)

sum1 =  23992
sum2 =  222
same1 =  23988
same2 =  1
all =  24214
all2 =  24214
